In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import Sequence
import pickle

In [4]:
#Load the text file
with open(r"D:\HP\Downloads\alllines.txt",'r') as file:
    text=file.read()
#Tokenize the text
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
#Convert text into sequences of words
sequences=tokenizer.texts_to_sequences([text])[0]
#Define sequence length
sequence_length=5
#Data generator class
class DataGenerator(Sequence):
    def __init__(self,sequences,batch_size,sequence_length,vocab_size):
        self.sequences=sequences
        self.batch_size=batch_size
        self.sequence_length=sequence_length
        self.vocab_size=vocab_size
        self.indices=np.arange(sequence_length,len(sequences))
    def __len__(self):
        return int(np.floor(len(self.indices)/self.batch_size))
    def __getitem__(self,index):
        batch_indices=self.indices[index * self.batch_size:(index+1) * self.batch_size]
        X=np.zeros((self.batch_size,self.sequence_length),dtype=np.int32)
        y=np.zeros((self.batch_size,self.vocab_size),dtype=np.float32)
        for i,idx in enumerate(batch_indices):
            X[i]=self.sequences[idx-self.sequence_length:idx]
            y[i,self.sequences[idx]]=1
        return X,y

vocab_size=len(tokenizer.word_index) + 1 
batch_size=64

#Instantiate the generator
data_gen=DataGenerator(sequences,batch_size=batch_size,sequence_length=sequence_length,vocab_size=vocab_size)

#Define the LSTM model
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=100,input_length=sequence_length),
    tf.keras.layers.LSTM(150,return_sequences=False),
    tf.keras.layers.Dense(vocab_size,activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#Train the model using the generator
model.fit(data_gen, epochs=10)

Epoch 1/10


D:\HP\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


    1/12831 ━━━━━━━━━━━━━━━━━━━━ 12:17:18 3s/step - accuracy: 0.0000e+00 - loss: 10.1495

D:\HP\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


12831/12831 ━━━━━━━━━━━━━━━━━━━━ 953s 74ms/step - accuracy: 0.0582 - loss: 6.7359
Epoch 2/10
12831/12831 ━━━━━━━━━━━━━━━━━━━━ 959s 75ms/step - accuracy: 0.1080 - loss: 5.8738
Epoch 3/10
12831/12831 ━━━━━━━━━━━━━━━━━━━━ 980s 76ms/step - accuracy: 0.1211 - loss: 5.6106
Epoch 4/10
12831/12831 ━━━━━━━━━━━━━━━━━━━━ 1139s 89ms/step - accuracy: 0.1316 - loss: 5.4011
Epoch 5/10
12831/12831 ━━━━━━━━━━━━━━━━━━━━ 1030s 80ms/step - accuracy: 0.1420 - loss: 5.2195
Epoch 6/10
12831/12831 ━━━━━━━━━━━━━━━━━━━━ 1055s 82ms/step - accuracy: 0.1517 - loss: 5.0651
Epoch 7/10
12831/12831 ━━━━━━━━━━━━━━━━━━━━ 1073s 84ms/step - accuracy: 0.1638 - loss: 4.9019
Epoch 8/10
12831/12831 ━━━━━━━━━━━━━━━━━━━━ 1087s 85ms/step - accuracy: 0.1758 - loss: 4.7663
Epoch 9/10
12831/12831 ━━━━━━━━━━━━━━━━━━━━ 1100s 86ms/step - accuracy: 0.1875 - loss: 4.6392
Epoch 10/10
12831/12831 ━━━━━━━━━━━━━━━━━━━━ 1120s 87ms/step - accuracy: 0.1995 - loss: 4.5180


In [ ]:
# Load the saved model
model = tf.keras.models.load_model(r"D:\HP\Downloads\models\shakespeare_lstm_model.h5")

# Load the saved tokenizer
with open(r"D:\HP\Downloads\models\tokenizer.pickle", 'rb') as handle:
    tokenizer = pickle.load(handle)

# Load the text file (Shakespeare plays)
with open(r"D:\HP\Downloads\alllines.txt", 'r') as file:
    text = file.read()

# Tokenize the text
sequences = tokenizer.texts_to_sequences([text])[0]

# Define sequence length
sequence_length = 5

# Data generator class
class DataGenerator(Sequence):
    def __init__(self, sequences, batch_size, sequence_length, vocab_size):
        self.sequences = sequences
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.indices = np.arange(sequence_length, len(sequences))
    
    def __len__(self):
        # Number of batches per epoch
        return int(np.floor(len(self.indices) / self.batch_size))
    
    def __getitem__(self, index):
        # Get a batch of indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        
        # Initialize X and y arrays
        X = np.zeros((self.batch_size, self.sequence_length), dtype=np.int32)
        y = np.zeros((self.batch_size, self.vocab_size), dtype=np.float32)
        
        # Generate data for the batch
        for i, idx in enumerate(batch_indices):
            X[i] = self.sequences[idx - self.sequence_length:idx]
            y[i, self.sequences[idx]] = 1  # One-hot encoding
        
        return X, y

# Parameters
vocab_size = len(tokenizer.word_index) + 1  # Number of unique words
batch_size = 64  # Example batch size

# Instantiate the generator
data_gen = DataGenerator(sequences, batch_size=batch_size, sequence_length=sequence_length, vocab_size=vocab_size)


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model using the generator
model.fit(data_gen, epochs=20)  # Adjust epochs as necessary
import pickle

# After training the model, save it as a .h5 file
model.save(r"D:\HP\Downloads\models\shakespeare_lstm_model.keras")

# Save the tokenizer as a pickle file
with open(r"D:\HP\Downloads\models\tokenizer.pickle", 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
model.save(r"D:\HP\Downloads\models\shakespeare_lstm_model.h5")

Epoch 1/20
    1/12831 ━━━━━━━━━━━━━━━━━━━━ 8:09:39 2s/step - accuracy: 0.2031 - loss: 3.8731

D:\HP\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


12831/12831 ━━━━━━━━━━━━━━━━━━━━ 930s 72ms/step - accuracy: 0.3486 - loss: 3.4563
Epoch 2/20
12831/12831 ━━━━━━━━━━━━━━━━━━━━ 934s 73ms/step - accuracy: 0.3576 - loss: 3.3831
Epoch 3/20
12831/12831 ━━━━━━━━━━━━━━━━━━━━ 4120s 321ms/step - accuracy: 0.3621 - loss: 3.3396
Epoch 4/20
12831/12831 ━━━━━━━━━━━━━━━━━━━━ 940s 73ms/step - accuracy: 0.3659 - loss: 3.3140
Epoch 5/20
12831/12831 ━━━━━━━━━━━━━━━━━━━━ 928s 72ms/step - accuracy: 0.3669 - loss: 3.3054
Epoch 6/20
12831/12831 ━━━━━━━━━━━━━━━━━━━━ 932s 73ms/step - accuracy: 0.3701 - loss: 3.2806
Epoch 7/20
12831/12831 ━━━━━━━━━━━━━━━━━━━━ 937s 73ms/step - accuracy: 0.3723 - loss: 3.2690
Epoch 8/20
12831/12831 ━━━━━━━━━━━━━━━━━━━━ 936s 73ms/step - accuracy: 0.3721 - loss: 3.2625
Epoch 9/20
12831/12831 ━━━━━━━━━━━━━━━━━━━━ 939s 73ms/step - accuracy: 0.3732 - loss: 3.2609
Epoch 10/20
12831/12831 ━━━━━━━━━━━━━━━━━━━━ 944s 74ms/step - accuracy: 0.3737 - loss: 3.2550
Epoch 11/20
12831/12831 ━━━━━━━━━━━━━━━━━━━━ 951s 74ms/step - accuracy: 0.3750

In [ ]:
# Optionally, make predictions with the loaded model 
def predict_next_word(model, tokenizer, seed_text, sequence_length):
    tokenized_input = tokenizer.texts_to_sequences([seed_text])[0]
    padded_input = pad_sequences([tokenized_input], maxlen=sequence_length, padding='pre')
    predicted = model.predict(padded_input, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=-1)[0]
    predicted_word = tokenizer.index_word[predicted_word_index]
    return predicted_word

# Example usage of prediction
seed_text = "to be or not to be a"
predicted_word = predict_next_word(model, tokenizer, seed_text, sequence_length)
print(f"Next word prediction: {predicted_word}")